<a href="https://colab.research.google.com/github/Drahtesel/Credit_Scoring_Prototypes/blob/main/Datenvorbereitung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ucimlrepo

# Datenvorbereitung

In [ ]:
# ============================================================
# German Credit Data – Datenvorbereitung
# ============================================================
# Dieses Notebook:
# Lädt das UCI German Credit Data Set
# Benennt Spalten verständlich um
# Ersetzt kategoriale Codes (A11, A12, …) durch Klartext
# Kodiert kategoriale Variablen durch One-Hot-Encoded (OHE)
# Skaliert numerische Variablen basierend auf Min-Max Normalisierung
# Passt die Zielwerte von [1,2] auf [0,1] an
# Erstellt Kopien des Datensatz mit begrenzter Merkmalszahl
# Speichert die vorbereiteten Daten im Drive als csv
# ============================================================

# --- Imports ---
from ucimlrepo import fetch_ucirepo
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import mutual_info_classif, chi2
from collections import Counter
from google.colab import drive
import os

# ============================================================
# Google Drive einbinden
# ============================================================
drive.mount('/content/drive')

output_dir = '/content/drive/MyDrive/Thesis/Daten/DataPreprocess/german_credit_prepared'
os.makedirs(output_dir, exist_ok=True)

# ============================================================
# Datensatz laden
# ============================================================
df = fetch_ucirepo(id=144)  # German Credit Data
X = df.data.features.copy()
y = df.data.targets.copy()

# ============================================================
# Spalten umbenennen (aus Attribut1–20 → verständliche Namen)
# ============================================================
column_names = {
    'Attribute1':  'checking_account',
    'Attribute2':  'duration',
    'Attribute3':  'credit_history',
    'Attribute4':  'purpose',
    'Attribute5':  'amount',
    'Attribute6':  'savings_account',
    'Attribute7':  'employment_since',
    'Attribute8':  'installment_rate',
    'Attribute9':  'personal_status_sex',
    'Attribute10': 'other_debtors',
    'Attribute11': 'residence_since',
    'Attribute12': 'property_status',
    'Attribute13': 'age',
    'Attribute14': 'other_installment_plans',
    'Attribute15': 'housing',
    'Attribute16': 'existing_credits',
    'Attribute17': 'job',
    'Attribute18': 'dependents',
    'Attribute19': 'telephone',
    'Attribute20': 'foreign_worker'
}
X.rename(columns=column_names, inplace=True)
y.rename(columns={'class': 'credit_risk'}, inplace=True)

# ============================================================
# Mapping der kategorialen Codes zu Klartext
# ============================================================
checking_account_status = {
    'A11': '< 0 DM', 'A12': '0 <= ... < 200 DM', 'A13': '>= 200 DM / salary assigned', 'A14': 'no checking account'}
credit_history = {
    'A30': 'no credits / all paid', 'A31': 'all credits at this bank paid',
    'A32': 'existing credits paid till now', 'A33': 'delayed payment in past', 'A34': 'critical / other credits exist'}
purpose = {
    'A40': 'car (new)', 'A41': 'car (used)', 'A42': 'furniture / equipment', 'A43': 'radio / television',
    'A44': 'domestic appliances', 'A45': 'repairs', 'A46': 'education', 'A47': 'vacation (?)',
    'A48': 'retraining', 'A49': 'business', 'A410': 'others'}
savings_account = {
    'A61': '< 100 DM', 'A62': '100 <= ... < 500 DM', 'A63': '500 <= ... < 1000 DM', 'A64': '>= 1000 DM',
    'A65': 'unknown / no savings'}
employment_since = {
    'A71': 'unemployed', 'A72': '< 1 year', 'A73': '1 <= ... < 4 years', 'A74': '4 <= ... < 7 years', 'A75': '>= 7 years'}
personal_status_sex = {
    'A91': 'male: divorced/separated', 'A92': 'female: div/sep/married',
    'A93': 'male: single', 'A94': 'male: married/widowed', 'A95': 'female: single'}
other_debtors = {'A101': 'none', 'A102': 'co-applicant', 'A103': 'guarantor'}
property_status = {'A121': 'real estate', 'A122': 'building society / insurance',
                   'A123': 'car or other', 'A124': 'unknown / no property'}
other_installment_plans = {'A141': 'bank', 'A142': 'stores', 'A143': 'none'}
housing = {'A151': 'rent', 'A152': 'own', 'A153': 'for free'}
job = {'A171': 'unemployed / non-resident', 'A172': 'unskilled - resident',
       'A173': 'skilled employee / official', 'A174': 'management / self-employed / high skill'}
telephone = {'A191': 'none', 'A192': 'yes, registered'}
foreign_worker = {'A201': 'yes', 'A202': 'no'}

# Alle Mappings anwenden
X['checking_account'] = X['checking_account'].map(checking_account_status)
X['credit_history'] = X['credit_history'].map(credit_history)
X['purpose'] = X['purpose'].map(purpose)
X['savings_account'] = X['savings_account'].map(savings_account)
X['employment_since'] = X['employment_since'].map(employment_since)
X['personal_status_sex'] = X['personal_status_sex'].map(personal_status_sex)
X['other_debtors'] = X['other_debtors'].map(other_debtors)
X['property_status'] = X['property_status'].map(property_status)
X['other_installment_plans'] = X['other_installment_plans'].map(other_installment_plans)
X['housing'] = X['housing'].map(housing)
X['job'] = X['job'].map(job)
X['telephone'] = X['telephone'].map(telephone)
X['foreign_worker'] = X['foreign_worker'].map(foreign_worker)

# ============================================================
# One-Hot-Encoding der kategorialen Variablen
# ============================================================
categorical_cols = [
    'checking_account', 'credit_history', 'purpose', 'savings_account',
    'employment_since', 'personal_status_sex', 'other_debtors',
    'property_status', 'other_installment_plans', 'housing',
    'job', 'telephone', 'foreign_worker'
]
# drop_first reduziert die Anzahl an Merkmalen ohne Information zu verlieren
X = pd.get_dummies(X, columns=categorical_cols, drop_first=True)

# ============================================================
# Skalierung der numerischen Variablen basierend auf Min-Max Normalisierung
# ============================================================
numeric_cols = ['duration', 'amount', 'installment_rate',
                'residence_since', 'age', 'existing_credits', 'dependents']

# --- Temporäre Skalierung für Chi² mit MinMaxScaler von 0 bis 1 ---
X_FS = X.copy()
scaler_temp = MinMaxScaler(feature_range=(0, 1))
X_FS[numeric_cols] = scaler_temp.fit_transform(X_FS[numeric_cols])

# --- Eigentliche Skalierung mit MinMaxScaler von -1 bis 1 ---
scaler = MinMaxScaler(feature_range=(-1, 1))
X[numeric_cols] = scaler.fit_transform(X[numeric_cols])

# ============================================================
# Label richtig kodieren
# ============================================================

y = y.squeeze()
print(type(y), y.shape)

# korrektes Mapping der Zielwerte
y = y.replace({1: 0, 2: 1}).astype(int)

X = X.reset_index(drop=True)
y = pd.Series(y).reset_index(drop=True)
assert not y.isna().any(), "y enthält NaN!"

# --- Daten bereinigen für XGBoost --------------------------------
X.columns = (
    X.columns
    .str.strip()
    .str.replace(" ", "_", regex=False)
    .str.replace("/", "_", regex=False)
    .str.replace("<", "lt", regex=False)
    .str.replace(">", "gt", regex=False)
    .str.replace("[", "_", regex=False)
    .str.replace("]", "_", regex=False)
)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
<class 'pandas.core.series.Series'> (1000,)


In [ ]:
# ============================================================
# Aufbereitete Daten im Drive speichern
# ============================================================
X.to_csv(f'{output_dir}/X_prepared.csv', index=False)
y.to_csv(f'{output_dir}/y_labels.csv', index=False)

print("Dateien gespeichert.")
print(f"X Shape: {X.shape}, y Shape: {y.shape}")

Dateien gespeichert.
X Shape: (1000, 48), y Shape: (1000,)


# Feature Selection durch Chi²

In [ ]:
# ============================================================
# Im folgenden werden die 15 besten Features herausgesucht
#     - Für jede gewünschte Merkmalsanzahl ausgeführt
# ============================================================

# --- Chi² Scores über alle OHE-Spalten ---
chi2_scores, _ = chi2(X_FS, y)

fs_df = pd.DataFrame({
    "Feature": X_FS.columns,
    "Chi2": chi2_scores
})

# ---- 2) Basisnamen extrahieren ----
fs_df["Base"] = fs_df["Feature"].apply(lambda x: x.split("_")[0])

# ---- 3) Score pro Basisname aggregieren (max pro Gruppe) ----
base_scores = fs_df.groupby("Base")["Chi2"].max().reset_index()

# ---- 4) Top-n Basisvariablen nach Chi² auswählen ----
top_n_bases = base_scores.sort_values("Chi2", ascending=False).head(15) # Anzahl gewünschter Merkmale
top_n = top_n_bases["Base"].tolist()

print("Top n Basisvariablen (Chi²):")
print(top_n)

# ---- 5) Alle zugehörigen One-Hot-Spalten auswählen ----
cols_to_keep = [
    col for col in X.columns
    if any(col.startswith(base + "_") or col == base for base in top_n)
]

# ---- 6) Filter anwenden ----
X_filtered = X[cols_to_keep]

# ---- 7) Vollständig entfernte Basisvariablen melden ----
removed_cols = [col for col in X.columns if col not in cols_to_keep]
removed_bases = sorted(
    {col.split("_")[0] for col in removed_cols} - set(top_n)
)

print("\nKomplett entfernte Basisvariablen:")
print(removed_bases)

# ---- 8) Daten überschreiben ----
X = X_filtered.copy()

# ---- 9) Exportieren ----
X.to_csv(f'{output_dir}/X_prepared_FS_15.csv', index=False)
y.to_csv(f'{output_dir}/y_labels_FS_15.csv', index=False)

print("Feature Selection angewendet und Dateien gespeichert.")
print(f"X Shape nach FS: {X.shape}, y Shape: {y.shape}")


Top n Basisvariablen (Chi²):
['checking', 'credit', 'savings', 'property', 'employment', 'purpose', 'housing', 'duration', 'amount', 'personal', 'other', 'age', 'installment', 'telephone', 'existing']

Komplett entfernte Basisvariablen:
['dependents', 'foreign', 'job', 'residence']
Feature Selection angewendet und Dateien gespeichert.
X Shape nach FS: (1000, 42), y Shape: (1000,)
